In [2]:
import pandas as pd
import numpy as np

In [3]:
mdf = pd.read_csv('data/movement_df_with_ld.csv', index_col=0)
# convert the NaNs in the "continent_code" to string "NA"
mdf['continent_code'] = mdf['continent_code'].fillna('NA')

mdf['move_time'] = pd.to_datetime(mdf['move_time'])

bins = list(range(1960, 2025, 5))
labels = [f'{i}-{i+4}' for i in bins[:-1]]

bins

[1960, 1965, 1970, 1975, 1980, 1985, 1990, 1995, 2000, 2005, 2010, 2015, 2020]

In [4]:
mdf['time_slot']  = pd.cut(mdf['move_time'].dt.year, bins=bins, labels=labels, right=False)

movement_counts = mdf.groupby(['prev_ins', 'institution_id', 'time_slot', 'level_distance'], observed = True).size()

movement_counts

/var/folders/5z/8hzvmhln5p36t4tdqc9c52z40000gn/T/ipykernel_84840/2841782303.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  movement_counts = mdf.groupby(['prev_ins', 'institution_id', 'time_slot', 'level_distance']).size()


: 

In [ ]:
def calculate_simulated_movement(row, df):
    time_slot = row['time_slot']
    prev_ins = row['prev_ins']
    institution_id = row['institution_id']

    total_movements = df[df['time_slot'] == time_slot].shape[0]
    outflow = df[(df['prev_ins'] == prev_ins) & (df['time_slot'] == time_slot)].shape[0]
    inflow = df[(df['institution_id'] == institution_id) & (df['time_slot'] == time_slot)].shape[0]

    return (outflow * inflow) / total_movements if total_movements != 0 else 0
